# 3. 데이터 변환 – 구간화, 정규화

## 구간화, 정규화
1. 구간화 : 연속형 데이터의 범주화
    - 지정 길이 기반 구간 : 데이터 범위를 사용자의 기준대로 정의하고 구간화
    - 분포 기반 구간 : 각 데이터가 모든 구간 내 동일한 개수로 구분되도록 구간화<br><br>
2. 정규화 : 연속형 데이터들의 관측 간격 조정 (스케일링)
    - 최대-최소 정규화 : 변수 안의 값이 [0,1] 구간의 값을 갖도록 구성하고, 데이터 군 내에서 특정 데이터가 가지는 위치를 보고자 할 때 사용
    - z-score 정규화 : 특정 데이터가 평균을 기준으로 얼마나 떨어져 있는지 파악하고 측정 스케일이 다른 경우의 데이터를 비교하는데 사용<br><br>
3. 어느 방안이 더 나은가?
    - 상황에 따라 달라지며, 어느 방안이 더 낫다기보다는 정규화 수행과 수행하지 않은 결과의 차이가 크다는 것이 중요한 포인트이다
    - 연속형 데이터를 기계학습 모델링에 적용하기 위해서는 변수 간의 상대적 크기 차이를 제거할 필요가 반드시 존재함<br><br>

## 실습 내용 요약
- 예제 데이터 활용 기반 구간화/정규화 방안 실습



In [4]:
import numpy as np
import pandas as pd

In [5]:
# 데이터 로딩 및 개요 확인
click_data = pd.read_csv("./data/click.csv")
click_data.head(15)

,category,journal,num_click
0,eng,D,164
1,tra,B,762
2,eng,D,220
3,tra,B,639
4,spo,A,894
5,soc,E,348
6,pol,C,521
7,spo,E,563
8,fin,B,250
9,pol,A,620


In [6]:
# 데이터 개요 파악
click_data.info()

# 데이터 copy
click_copy = click_data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17734 entries, 0 to 17733
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   category   17734 non-null  object
 1   journal    17734 non-null  object
 2   num_click  17734 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 415.8+ KB


## 3-1. 구간화

In [7]:
click_copy.describe()

,num_click
count,17734.000000
mean,543.845495
std,5619.881420
min,0.000000
25%,325.000000
50%,459.000000
75%,597.000000
max,433992.000000


### 시나리오 가정
- 기사의 선호도를 파악하고자 하는데, 단순 클릭수가 절대적인 선호도 척도가 될 수 있을까?
- NO, 기사의 카테고리, 업로드 시간 등 클릭을 유도하는 다양한 요소들이 있을 것이기 때문<br><br>

- 따라서 구간화를 통해, 새로운 의미를 지니는 변수가 필요하며 목적에 맞는 변수 설정이 중요
- 클릭수를 기준으로 범주로 변환해 각 범주 내에서 의미 있는 데이터 처리를 생각해 볼 수 있다. (도메인 지식 검토 필요)

### 1) 지정 길이 기반 구간화

In [8]:
# 클릭수를 기반으로 범주형 구간화 (3개 구간 설정)
# cut 함수 활용 기반 구간 설정 (사용자 기준)
bins = [0, 100, 500, np.max(click_copy['num_click'])]
names = ['low', 'medium', 'high']

click_copy['preference'] = pd.cut(click_copy['num_click'], bins, labels=names)

In [9]:
# 구간 결과 확인
click_copy['preference'].value_counts()

medium    9582
high      7482
low        464
Name: preference, dtype: int64

In [10]:
click_copy.head(15)

,category,journal,num_click,preference
0,eng,D,164,medium
1,tra,B,762,high
2,eng,D,220,medium
3,tra,B,639,high
4,spo,A,894,high
5,soc,E,348,medium
6,pol,C,521,high
7,spo,E,563,high
8,fin,B,250,medium
9,pol,A,620,high


### 2) 분포 기반 구간화

In [11]:
# 분포 기반 구간화
# qcut의 경우 동일한 관측치가 배치되도록 구간을 분할
# 따라서 구간수로 나누게 됨
# 구간수 = n
n = 3
click_copy['pref_qcut'] = pd.qcut(click_copy['num_click'], n, labels=names)
print(click_copy['pref_qcut'].value_counts())

medium    5921
low       5920
high      5893
Name: pref_qcut, dtype: int64


**qcut 실행 시 주로 발생할 수 있는 오류들**
- 데이터가 극단적으로 skewed 되어있는 경우, 데이터의 각 구간이 동일할 수 없을 수 있다.
    - ex) [1, 1, 1, 1, 1, 3, 8, 10] 이라는 구간은 분포 기반 구간화가 어려울 수 있다.
- 이러한 경우에는 분포 기반 구간화(qcut)보다는 사용자 기준 기반 범주화 혹은 구간 개수를 변화하는 것을 고려할 수 있다

## 3-2. 정규화

### 1) 최대-최소 정규화 (Min-Max-scaling)

In [13]:
# 변수 값을 0에서 1사이 값으로 변환
# 함수는 sklearn 라이브러리에 잘 구현되어 있다.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# 데이터 copy
click_copy = click_data.copy()

# 변환
click_copy['minmax_values'] = scaler.fit_transform(click_copy[['num_click']])
click_copy.head(15)

,category,journal,num_click,minmax_values
0,eng,D,164,0.000378
1,tra,B,762,0.001756
2,eng,D,220,0.000507
3,tra,B,639,0.001472
4,spo,A,894,0.002060
5,soc,E,348,0.000802
6,pol,C,521,0.001200
7,spo,E,563,0.001297
8,fin,B,250,0.000576
9,pol,A,620,0.001429


In [14]:
# 정말로 [0,1] 사이에서 잘 정규화가 되었는지 확인, min=0, max=1로 잘 변환되었다.
click_copy.describe()

,num_click,minmax_values
count,17734.000000,17734.000000
mean,543.845495,0.001253
std,5619.881420,0.012949
min,0.000000,0.000000
25%,325.000000,0.000749
50%,459.000000,0.001058
75%,597.000000,0.001376
max,433992.000000,1.000000


### 2) Z-Score 정규화 (Z-Score Normaliztion)

In [15]:
# 변수를 평균이 0이고 표준편차가 1인 표준정규분포표로 변환
# 함수는 sklearn 라이브러리에 잘 구현되어 있다.
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

# 변환
click_copy['std_values'] = std_scaler.fit_transform(click_copy[['num_click']])
click_copy.head(15)

,category,journal,num_click,minmax_values,std_values
0,eng,D,164,0.000378,-0.067591
1,tra,B,762,0.001756,0.038819
2,eng,D,220,0.000507,-0.057627
3,tra,B,639,0.001472,0.016932
4,spo,A,894,0.002060,0.062308
5,soc,E,348,0.000802,-0.034850
6,pol,C,521,0.001200,-0.004065
7,spo,E,563,0.001297,0.003408
8,fin,B,250,0.000576,-0.052288
9,pol,A,620,0.001429,0.013551


In [17]:
# 데이터 요약
click_copy.describe()

,num_click,minmax_values,std_values
count,17734.000000,17734.000000,1.773400e+04
mean,543.845495,0.001253,-2.771019e-18
std,5619.881420,0.012949,1.000028e+00
min,0.000000,0.000000,-9.677443e-02
25%,325.000000,0.000749,-3.894240e-02
50%,459.000000,0.001058,-1.509781e-02
75%,597.000000,0.001376,9.458563e-03
max,433992.000000,1.000000,7.712981e+01
